# Main part of the code

#### Importing libraries

In [ ]:
#Importing necessary libraries
!pip install optuna
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
from torch.autograd.functional import jacobian as jac
from torch.func import jacfwd, vmap
from csv import writer
import optuna #for hyperparameter search

#Importing necessary scripts
from scripts.createDataset import getData, getDataLoaders
from scripts.utils import getBCs
from scripts.network import approximate_curve
from scripts.training import trainModel
from scripts.evaluateModel import 

In [ ]:
#We do this so Pytorch works in double precision
torch.set_default_dtype(torch.float32)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

#### Importing and organising data

We can create the dataset as follows:

Input : (q_1,q_2,v_1,v_2,s)
Output : (q(s),v(s))

With the trajectories that we have available, we can hence generate a dataset of size
N_elements+1 x N_samples

In [ ]:
#Importing and shuffling the trajectories

trajectories = np.loadtxt("generated_data.txt")
number_samples,number_components = trajectories.shape
#Randomize the order of the trajectories
indices = np.random.permutation(len(trajectories))
trajectories = trajectories[indices]

number_elements = int(number_components/4)-1
data_train, data_test = getData(number_elements,number_samples,trajectories)

#### Training the neural network iterating over the training batches

In [ ]:
def define_model(trial):
    normalize = trial.suggest_categorical("normalize",[True,False])
    act = trial.suggest_categorical("act",['tanh','sigmoid','sin','relu2'])
    nlayers = trial.suggest_int("n_layers", 1, 4)
    hidden_nodes = trial.suggest_int("hidden_nodes", 10, 100)
    correct_functional = trial.suggest_categorical("correct_functional",[True,False])

    model = approximate_curve(normalize,act, nlayers, hidden_nodes, correct_functional)
    return model

In [ ]:
def objective(trial):
    # Generate the model.
    model = define_model(trial)
    model.to(device);

    lr = trial.suggest_float("lr", 1e-4 , 1e-1, log=True)
    weight_decay = trial.suggest_float("weight_decay",0,5e-4)
    #optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    #optimizer = getattr(torch.optim, optimizer_name)(model.parameters(), lr=lr, weight_decay=weight_decay)

    #optimizer = torch.optim.LBFGS(model.parameters(),lr=lr, max_iter=100,tolerance_grad=1.e-10,
    #                                  tolerance_change=1.e-10, history_size=100, line_search_fn='strong_wolfe')
    optimizer = torch.optim.Adam(model.parameters(),lr=lr,weight_decay=weight_decay)

    criterion = nn.MSELoss()


    batch_size = 32 # trial.suggest_int("batch_size", 32, 64)

    trainloader, testloader = getDataLoaders(batch_size,data_train,data_test)

    print("Current test with :\n\n")
    for key, value in trial.params.items():
      print("    {}: {}".format(key, value))
    print("\n\n")

    epochs = 100
    loss = trainModel(number_elements,device,model,criterion,optimizer,epochs,trainloader)
    test_error = 100
    if not torch.isnan(loss):
      model.eval();

      def eval_model(s,q1,q2,v1,v2):
        s_ = torch.tensor([[s]],dtype=torch.float32).to(device)
        q1 = torch.from_numpy(q1.astype(np.float32)).reshape(1,-1).to(device)
        q2 = torch.from_numpy(q2.astype(np.float32)).reshape(1,-1).to(device)
        v1 = torch.from_numpy(v1.astype(np.float32)).reshape(1,-1).to(device)
        v2 = torch.from_numpy(v2.astype(np.float32)).reshape(1,-1).to(device)
        return model(s_,q1,q2,v1,v2).detach().cpu().numpy()[0]

      bcs = getBCs(trajectories)
      q1 = bcs["q1"]
      q2 = bcs["q2"]
      v1 = bcs["v1"]
      v2 = bcs["v2"]

      xx = np.linspace(0, 1, number_elements+1)
      res = np.zeros((50,2,len(xx)))

      for j in range(50):
          for i in range(len(xx)):
              res[j,:,i] = eval_model(xx[i],q1[j],q2[j],v1[j],v2[j])

      q_x_pred_torch = torch.from_numpy(res[:,0].astype(np.float32))
      q_x_true_torch = torch.from_numpy(trajectories[:50,np.arange(0,number_components,4)].astype(np.float32))

      q_y_pred_torch = torch.from_numpy(res[:,1].astype(np.float32))
      q_y_true_torch = torch.from_numpy(trajectories[:50,np.arange(1,number_components,4)].astype(np.float32))

      test_error = criterion(q_x_pred_torch,q_x_true_torch).item() + criterion(q_y_pred_torch,q_y_true_torch).item()

    #Saving the obtained results
    if trial.number == 0:
        labels = []
        for lab, _ in trial.params.items():
            labels.append(str(lab))
        labels.append("Test error")
        with open("savedResults/results.csv", "a") as f_object:
            writer_object = writer(f_object)
            writer_object.writerow(labels)
            f_object.close()

    results = []
    for _, value in trial.params.items():
        results.append(str(value))

    results.append(test_error)

    with open("savedResults/results.csv", "a") as f_object:
        writer_object = writer(f_object)
        writer_object.writerow(results)
        f_object.close()

    return test_error

In [ ]:
study = optuna.create_study(direction="minimize",study_name="Euler Elastica")
study.optimize(objective, n_trials=100)
print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))

#### Plot the obtained results

In [ ]:
model.eval();
plotTestResults(model,number_elements,number_components,trajectories)